In [9]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
    print("Set GOOGLE_API_KEY environment variable.")

In [10]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
print("Gemini-2.5-flash model initialized successfully.")

Gemini-2.5-flash model initialized successfully.


In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
print("Google Generative AI Embeddings initialized successfully.")

Google Generative AI Embeddings initialized successfully.


In [12]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
print("In-memory vector store created successfully.")

In-memory vector store created successfully.


In [ ]:
from langchain import hub
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Read the document.txt file
with open("document.txt", "r", encoding="utf-8") as file:
    content = file.read()

# Create a Document object
docs = [Document(
    page_content=content,
    metadata={
        "source": "document.txt",
        "type": "professional_profile",
        "author": "M Vashishta Varma"
    }
)]

print("Document loaded successfully.")


text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Load prompt from hub
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()
print("Application compiled successfully.")


Document loaded successfully.
Application compiled successfully.
Application compiled successfully.


In [14]:
# "What GitHub achievements does Vashishta have?"
# "How many repositories does Vashishta have?"
# "What is Vashishta's primary programming language?"
# "Explain the Huffman coding implementation"
# "What web technologies are used in the projects?"
# "How is the Flask application structured?"
# "What testing frameworks are mentioned?"
# "What are Vashishta's specialized areas?"
# "What kind of applications has Vashishta built?"
# "What databases has Vashishta worked with?"
# "What deployment technologies are mentioned?"

response = graph.invoke({"question": "What GitHub achievements does Vashishta have and how many repositories does he have?"})
print(response["answer"])

Vashishta has the GitHub achievements of YOLO, Pull Shark x2, and Quickdraw. He currently has 9 active repositories.
